In [73]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/nlp_class/all_book_titles.txt

File ‘all_book_titles.txt’ already there; not retrieving.



In [74]:
import nltk
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD


In [75]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mehmetdegirmenci/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mehmetdegirmenci/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mehmetdegirmenci/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [76]:
wordnet_lemmatizer = WordNetLemmatizer()

In [77]:
titles = (line.rstrip('\n') for line in open('all_book_titles.txt'))

In [78]:
stops = set(stopwords.words('english'))

In [79]:
stops = stops.union({'introduction', 'edition', 'series', 'application',
  'approach', 'card', 'access', 'package', 'plus', 'etext',
  'brief', 'vol', 'fundamental', 'guide', 'essential', 'printed',
  'third', 'second', 'fourth', 'volume'
})

In [80]:
def my_tokenizer(s):
    # tüm harfleri küçük harfe çevirdik
    s=s.lower()

    # metin kelimelere ayrılır tokenize edilir
    tokens = nltk.tokenize.word_tokenize(s)

    # 2 harfeten kısa olan kelimeler çıkartılır
    # to in on it an çok fazla bilgi taşımaz
    tokens = [t for t in tokens if len(t)>2]

    #her kök lemmatize edilir köklerine ayrılır
    # ilk vakitlerde yapmıştın hatırla
    tokens = [wordnet_lemmatizer.lemmatize(t)for t in tokens ]

    # önceden belirlenen stopword kelimelerine bakılır ve bu kelimeler atılır
    #Hem NLTK’nin listesi hem de özel domain-specific kelimeler burada filtrelenir.
    tokens = [t for t in tokens if t not in stops]

    # içinde rakam içeren bütün kelimeler çıkarılır "3rd" gibi gibi
    # bu kitap baskısıyla ilgili bütün kelimelerin çıkarılmasını sağlar
    tokens = [t for t in tokens if not any(c.isdigit() for c in t)]

    return tokens

# fonksiyonun amacı normalize eder kök bulur
# temizler stopword ve rakamlar gibi şeyler bulunmaz kısa kelimeler de olmaz
# tokenlere bölme işlemi de yapılmış olur sadece kök bulunur yani





In [81]:
vectorizer = CountVectorizer(binary = True ,tokenizer= my_tokenizer)


In [82]:
X = vectorizer.fit_transform(titles)
# burada aldığımız hata aslında warning yani tam olarak hata değil
# burada demek istediği sen zaten tokenizer olarak kendi tokenizerını oluşturdun
# benim token patternimi yok sayacağım demiş

/Users/mehmetdegirmenci/PyCharmMiscProject/.venv/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



In [83]:
index_word_map = vectorizer.get_feature_names_out()

In [84]:
X = X.T
# rowu terms olarak aldı , columnsu da documents olarak aldı
# X başlangıçta:
# satırlar = kitap başlıkları, sütunlar = kelimeler olan bir matristi.
# .T: Transpose (matrisi döndürür)
# Artık:
# satırlar = kelimeler
# sütunlar = kitap başlıkları

In [85]:
svd = TruncatedSVD(n_components=2)  #sparse matrisler için uygundur boyut indirme uygulamasıdır aslında

Z= svd.fit_transform(X) # her kelimeyi 2D düzlemde bir nokta olarak belli eder
#burada amaç yüksek boyutlu kelimleri 2 boyuta indirmek ve grafikte nokta olarak göstermek


In [86]:
!pip install plotly
# bu satır interaktif grafikler oluşturmayı sağlayan plotly kütüphanesini içerir

In [87]:
import plotly.express as px
import plotly.io as pio

In [88]:
fig = px.scatter(x=Z[:,0], y=Z[:,1])

fig.show()


In [89]:
print(X.shape)

(2131, 2373)


In [92]:
print(Z.shape)

(2131, 2)


In [91]:
fig = px.scatter(
    x=Z[:100, 0],
    y=Z[:100, 1],
    text=index_word_map[:100],
    size_max=60
)
fig.update_traces(textposition='top center')
fig.show()
fig.write_html('plot.html')

